# Naive Bayes Classifiers

We want to classify vectors of discrete value features, $\mathbf{x}\in\{1,\ldots,K\}^D$, where $K$ is the number of values for each feature, and $D$ is the number of features. If we use a generative approach, we will need to specify the class conditional distribution $p(\mathbf{x}|y=c)$.

The simplest approach is to assume that the features are **conditionally independent** given the class label. This means we can write the class conditional density as a product of one-dimensional densities:

$$
p(\mathbf{x}|y=c, \theta) = \prod_{j=1}^Dp(x_j|y=c,\theta_{jc})
$$

The resulting model is called a Naive Bayes Classifier (NBC). This has $O(CD)$ parameters, for $C$ classes and $D$ features.

## Model fitting
'Training' a Naive Bayes classifier usually means computing the MLE or MAP estimate for the parameters. We can also compute the full posterior $p(\theta|\mathcal{D})$

### MLE
the probability for a single data case is given by

$$
p(\mathbf{x}_i, y_i|\theta) = p(y_i|\pi)\prod_jp(x_{ij}|\theta_j)
$$